In [7]:
import pandas as pd
from scipy import stats
from statistics import mean
from datetime import datetime as DateTime
import yfinance as yf
import pytickersymbols as pts

In [8]:
yf.__version__

'0.1.62'

In [9]:
pts.__version__

'1.4.3'

## Acquiring information about stocks 

In [10]:
stock_data = pts.PyTickerSymbols()

eu50_stocks = stock_data.get_stocks_by_index('EURO STOXX 50')
us30_stocks = stock_data.get_stocks_by_index('DOW JONES')
uk100_stocks = stock_data.get_stocks_by_index('FTSE 100')
us100_stocks = stock_data.get_stocks_by_index('S&P 100')

market_stocks = list(eu50_stocks) + list(us30_stocks) + list(uk100_stocks) + list(us100_stocks)

excluded_sectors = ['Biotechnology & Medical Research','Online gambling','Pharmaceuticals','Healthcare','Managed Health care','Medical Equipment','Banking & Investment Services','Medical Equipment, Supplies & Distribution','Metals & Mining','Metal','Insurance','Commercial REITs','Specialized REITs','Financial services','Beverages','Food & Beverages','Real Estate','Casinos & Gaming','Fossil Fuels']

stocks_list = []
for market_stock in market_stocks:
    try:
        ignore = False
        for market_industry in market_stock['industries']:
            if(market_industry in excluded_sectors):
                ignore = True
                break
            
        if(ignore):
            continue

        stock_symbol = market_stock['symbol']
        symbols = market_stock['symbols']

        symbol_google = None
        symbol_yahoo = None
        for symbol in symbols:
            if(symbol['google'].endswith(f':{stock_symbol}')):
                symbol_google = symbol['google']
                symbol_yahoo = symbol['yahoo']
                break

        if(symbol_yahoo is None):
            continue

        item = {
            'Company': market_stock['name'],
            'Symbol': stock_symbol,
            'Symbol_Google': symbol_google,
            'Symbol_Yahoo': symbol_yahoo,
            'Country': market_stock['country'],
            'Indices': ', '.join(market_stock['indices']),
            'Sectors': ', '.join(market_stock['industries'])
        }

        stocks_list.append(item)
    except:
        pass

stocks = pd.DataFrame(stocks_list, columns=['Company','Symbol','Symbol_Google','Symbol_Yahoo','Country','Indices','Sectors'])
stocks

,Company,Symbol,Symbol_Google,Symbol_Yahoo,Country,Indices,Sectors
0,adidas AG,ADS,FRA:ADS,ADS.F,Germany,"DAX, EURO STOXX 50","Cyclical Consumer Products, Footwear, Textiles..."
1,BASF SE,BAS,FRA:BAS,BAS.F,Germany,"DAX, EURO STOXX 50","Diversified Chemicals, Basic Materials, Chemicals"
2,Bayerische Motoren Werke AG,BMW,FRA:BMW,BMW.F,Germany,"DAX, EURO STOXX 50","Automotive, Auto & Truck Manufacturers, Automo..."
3,Daimler AG,DAI,FRA:DAI,DAI.F,Germany,"DAX, EURO STOXX 50","Automotive, Auto & Truck Manufacturers, Automo..."
4,Deutsche Post AG,DPW,FRA:DPW,DPW.F,Germany,"DAX, EURO STOXX 50","Industrials, Air Freight & Courier Services, F..."
...,...,...,...,...,...,...,...
133,United Parcel Service Inc.,UPS,NYSE:UPS,UPS,United States,"S&P 100, S&P 500","Industrial Goods, Industrials, Aerospace & Def..."
134,salesforce.com Inc.,CRM,NYSE:CRM,CRM,United States,"DOW JONES, S&P 100, S&P 500","Technology, Software & IT Services, IT Service..."
135,Dow Inc.,DOW,NYSE:DOW,DOW,United States,"DOW JONES, S&P 100, S&P 500",Chemicals
136,General Dynamics Corporation,GD,NYSE:GD,GD,United States,"S&P 100, S&P 500","Aerospace, Defense"


In [11]:
# Some stocks are listed on more than the stock index.
stocks.set_index('Symbol',inplace=True)
stocks = stocks[~stocks.index.duplicated(keep='first')]
stocks

,Company,Symbol_Google,Symbol_Yahoo,Country,Indices,Sectors
Symbol,,,,,,
ADS,adidas AG,FRA:ADS,ADS.F,Germany,"DAX, EURO STOXX 50","Cyclical Consumer Products, Footwear, Textiles..."
BAS,BASF SE,FRA:BAS,BAS.F,Germany,"DAX, EURO STOXX 50","Diversified Chemicals, Basic Materials, Chemicals"
BMW,Bayerische Motoren Werke AG,FRA:BMW,BMW.F,Germany,"DAX, EURO STOXX 50","Automotive, Auto & Truck Manufacturers, Automo..."
DAI,Daimler AG,FRA:DAI,DAI.F,Germany,"DAX, EURO STOXX 50","Automotive, Auto & Truck Manufacturers, Automo..."
DPW,Deutsche Post AG,FRA:DPW,DPW.F,Germany,"DAX, EURO STOXX 50","Industrials, Air Freight & Courier Services, F..."
...,...,...,...,...,...,...
TGT,Target Corporation,NYSE:TGT,TGT,United States,"S&P 100, S&P 500","Diversified Retail, Retailers, Discount Stores..."
UNP,Union Pacific Corporation,FRA:UNP,UNP.F,United States,"S&P 100, S&P 500","Ground Freight & Logistics, Industrials, Freig..."
UPS,United Parcel Service Inc.,NYSE:UPS,UPS,United States,"S&P 100, S&P 500","Industrial Goods, Industrials, Aerospace & Def..."


## Transforming data for value calculation 

In [13]:
# Defining stocks value scoring dataframe

df_columns = [
    'Company',
    'Symbol',
    'Symbol_Google',
    'Symbol_Yahoo',
    'Description',
    'Country',        
    'Indices',
    'Sectors',
    'MarketCap',
    'Price',
    'Currency',
    'PriceToEarningsRatio',
    'PriceToEarningsPercentile',
    'PriceToSalesRatio',
    'PriceToSalesPercentile',
    'EV_EBITDA',
    'EV_EBITDAPercentile',
    'EV_R',
    'EV_RPercentile',
    'Score'
]

so_dataframe = pd.DataFrame(columns = df_columns)

# Getting fundamental data
i = 0
total_stocks = len(stocks)
errors = 0
for index, stock in stocks.iterrows():
    i+=1
    stock_name = stock['Company']
    symbol = index

    symbol_google = stock['Symbol_Google']
    symbol_yahoo = stock['Symbol_Yahoo']

    print(f'({i}/{total_stocks}) | {symbol} (Google: {symbol_google}, Yahoo: {symbol_yahoo}) | {stock_name}')
    print(f'Retreiving fundamental data from yfinance...')

    try:
        yf_ticker = yf.Ticker(symbol_yahoo)
        so_dataframe = so_dataframe.append(
            pd.Series([
                stock_name,
                symbol,
                symbol_google,
                symbol_yahoo,
                yf_ticker.info['longBusinessSummary'],
                stock['Country'],
                stock['Indices'],
                stock['Sectors'],
                yf_ticker.info['marketCap'],
                yf_ticker.info['regularMarketPrice'],
                yf_ticker.info['financialCurrency'],
                yf_ticker.info['trailingPE'],
                'N/A',
                yf_ticker.info['priceToSalesTrailing12Months'],
                'N/A',
                yf_ticker.info['enterpriseToEbitda'],
                'N/A',
                yf_ticker.info['enterpriseToRevenue'],
                'N/A',
                'N/A'
            ], index = df_columns),
            ignore_index = True)

    except Exception as error:
        print('ERROR:', error)
        errors+=1
        print('An error occurred while trying to get the information!')
        
    print()

initial_count = len(so_dataframe)

# Dealing With Missing Data
so_dataframe = so_dataframe.dropna()

(1/116) | ADS (Google: FRA:ADS, Yahoo: ADS.F) | adidas AG
Retreiving fundamental data from yfinance...

(2/116) | BAS (Google: FRA:BAS, Yahoo: BAS.F) | BASF SE
Retreiving fundamental data from yfinance...
ERROR: 'trailingPE'
An error occurred while trying to get the information!

(3/116) | BMW (Google: FRA:BMW, Yahoo: BMW.F) | Bayerische Motoren Werke AG
Retreiving fundamental data from yfinance...

(4/116) | DAI (Google: FRA:DAI, Yahoo: DAI.F) | Daimler AG
Retreiving fundamental data from yfinance...

(5/116) | DPW (Google: FRA:DPW, Yahoo: DPW.F) | Deutsche Post AG
Retreiving fundamental data from yfinance...

(6/116) | DTE (Google: FRA:DTE, Yahoo: DTE.F) | Deutsche Telekom AG
Retreiving fundamental data from yfinance...

(7/116) | EOAN (Google: FRA:EOAN, Yahoo: EOAN.F) | E.ON SE
Retreiving fundamental data from yfinance...

(8/116) | LIN (Google: FRA:LIN, Yahoo: LIN.F) | Linde PLC
Retreiving fundamental data from yfinance...

(9/116) | SAP (Google: FRA:SAP, Yahoo: SAP.F) | SAP SE
Ret

In [14]:
# Calculating percentiles
print('Calculating Percentiles...')

metrics = {
    'PriceToEarningsRatio': 'PriceToEarningsPercentile',
    'PriceToSalesRatio': 'PriceToSalesPercentile',
    'EV_EBITDA':'EV_EBITDAPercentile',
    'EV_R':'EV_RPercentile'
}

for row in so_dataframe.index:
    for metric in metrics.keys():
        if(metric == 'PriceToSalesRatio' or metric == 'EV_R'):
            so_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(1 / so_dataframe[metric], 1 / so_dataframe.loc[row, metric])
        else:
            so_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(so_dataframe[metric], so_dataframe.loc[row, metric])

# Scoring stocks
print('Scoring Stocks...')
for row in so_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(so_dataframe.loc[row, metrics[metric]])
    so_dataframe.loc[row, 'Score'] = mean(value_percentiles)

# Sorting and selecting
result_df = so_dataframe[so_dataframe['Score'] >= 50].copy()
result_df.sort_values(by = 'Score', inplace = True, ascending=False)

skipped_count = initial_count - len(result_df)
now = DateTime.now()

print()
print(f'Scoring executed successfully at {now.hour}:{now.minute} | {total_stocks} listed | {errors} not found | {skipped_count} incomplete | {len(result_df)} selected')

Calculating Percentiles...
Scoring Stocks...

Scoring executed successfully at 12:52 | 116 listed | 20 not found | 43 incomplete | 53 selected


In [15]:
result_df

,Company,Symbol,Symbol_Google,Symbol_Yahoo,Description,Country,Indices,Sectors,MarketCap,Price,Currency,PriceToEarningsRatio,PriceToEarningsPercentile,PriceToSalesRatio,PriceToSalesPercentile,EV_EBITDA,EV_EBITDAPercentile,EV_R,EV_RPercentile,Score
69,Costco Wholesale Corporation,COST,NASDAQ:COST,COST,"Costco Wholesale Corporation, together with it...",United States,"NASDAQ 100, S&P 100, S&P 500","Diversified Retail, Retailers, Discount Stores...",182487744512,412.370,USD,42.168930,73.6842,0.977768,83.1579,20.052,64.2105,0.932,92.6316,78.4211
58,Wm Morrison Supermarkets PLC,MRW,LON:MRW,MRW.L,Wm Morrison Supermarkets PLC operates retail s...,United Kingdom,FTSE 100,"Consumer Non-Cyclicals, Food & Drug Retailing,...",6383454208,264.900,GBP,66.225000,91.5789,0.362737,95.7895,11.203,22.1053,0.499,98.4211,76.9737
0,adidas AG,ADS,FRA:ADS,ADS.F,"adidas AG, together with its subsidiaries, des...",Germany,"DAX, EURO STOXX 50","Cyclical Consumer Products, Footwear, Textiles...",61758750720,315.950,EUR,64.400734,89.4737,2.979053,49.4737,31.882,83.1579,3.048,60,70.5263
61,JD Sports Fashion plc,JD,LON:JD,JD.L,JD Sports Fashion plc engages in the retail of...,United Kingdom,FTSE 100,Retail,9872699392,957.000,GBP,41.608696,71.5789,1.600814,69.4737,16.822,54.7368,1.709,75.7895,67.8947
93,United Parcel Service Inc.,UPS,NYSE:UPS,UPS,"United Parcel Service, Inc. provides letter an...",United States,"S&P 100, S&P 500","Industrial Goods, Industrials, Aerospace & Def...",187483979776,213.920,USD,36.122932,65.2632,2.094770,62.1053,21.231,69.4737,2.194,70.5263,66.8421
29,Walgreens Boots Alliance Inc.,WBA,NASDAQ:WBA,WBA,"Walgreens Boots Alliance, Inc. operates as a p...",United States,"DOW JONES, NASDAQ 100, S&P 100, S&P 500","Consumer Non-Cyclicals, Food & Drug Retailing,...",41009033216,47.410,USD,17.924385,16.8421,0.280553,98.9474,15.626,50.5263,0.560,95.7895,65.5263
42,Johnson Matthey PLC,JMAT,LON:JMAT,JMAT.L,Johnson Matthey Plc provides specialty chemica...,United Kingdom,FTSE 100,"Specialty Chemicals, Basic Materials, Chemicals",6119513088,3162.000,GBP,29.718044,48.4211,0.390449,94.7368,10.197,17.8947,0.432,100,65.2632
66,Amazon.com Inc.,AMZN,NASDAQ:AMZN,AMZN,"Amazon.com, Inc. engages in the retail sale of...",United States,"NASDAQ 100, S&P 100, S&P 500","Diversified Retail, Retailers, Department Stor...",1875752517632,3719.340,USD,70.761010,93.6842,4.475347,37.8947,32.918,84.2105,4.490,43.1579,64.7368
85,Ford Motor Company,F,NYSE:F,F,"Ford Motor Company designs, manufactures, mark...",United States,"S&P 100, S&P 500","Auto & Truck Manufacturers, Automobiles & Auto...",57798946816,14.480,USD,14.567404,11.5789,0.447873,92.6316,21.708,71.5789,1.422,82.1053,64.4737
59,Ferguson PLC,FERG,LON:FERG,FERG.L,Ferguson plc distributes plumbing and heating ...,United Kingdom,FTSE 100,Building materials,23175823360,10350.000,USD,34.147144,57.8947,1.042313,80,11.846,29.4737,1.018,89.4737,64.2105


In [16]:
with open("../dist/md.txt", "a", encoding='utf-8') as f:
    md_df = result_df[['Company','Symbol','Symbol_Yahoo', 'PriceToEarningsRatio','PriceToEarningsPercentile','PriceToSalesRatio','PriceToSalesPercentile','EV_EBITDA','EV_EBITDAPercentile','EV_R','EV_RPercentile','Score']]
    md_df.columns = ['Company','Symbol','Yahoo Symbol', 'P/E', 'P/E PC', 'P/S','P/S PC', 'EV/EBITDA','EV/EBITDA PC', 'EV/R','EV/R PC','Score']
    f.write(md_df.to_markdown(index=False))

In [17]:
writer = pd.ExcelWriter('../dist/market_stock_value_scoring.xlsx', engine='xlsxwriter')
result_df.to_excel(writer, sheet_name='Value Scoring', index = False)

# Defining Excel columns formats and templates
background_light_color = '#ffffff'
background_dark_color = '#eeeeee'
background_accent_color = '#0086CD'
black_color = '#000000'
white_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': black_color,
            'bg_color': background_light_color,
            'border': 0
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.000',
            'font_color': black_color,
            'bg_color': background_light_color,
            'border': 0
        }
    )

percentile_template = writer.book.add_format(
        {
            'num_format':'0.000',
            'font_color': black_color,
            'bg_color': background_dark_color,
            'border': 0
        }
    )

score_template = writer.book.add_format(
        {
            'num_format':'0.000',
            'font_color': white_color,
            'bg_color': background_accent_color,
            'border': 0
        }
    )

print('Saving in Excel...\n')

column_formats = {
    'A': ['Stock', string_template],
    'B': ['Symbol', string_template],
    'C': ['Symbol_Google', string_template],
    'D': ['Symbol_Yahoo', string_template],
    'E': ['Description', string_template],
    'F': ['Country', string_template],
    'G': ['Indices', string_template],
    'H': ['Sectors', string_template],
    'I': ['MarketCap', float_template],
    'J': ['Price', float_template],
    'K': ['Currency', string_template],
    'L': ['PriceToEarningsRatio', float_template],
    'M': ['PriceToEarningsPercentile', percentile_template],
    'N': ['PriceToSalesRatio', float_template],
    'O': ['PriceToSalesPercentile', percentile_template],
    'P': ['EV_EBITDA', float_template],
    'Q': ['EV_EBITDAPercentile', percentile_template],
    'R': ['EV_R', float_template],
    'S': ['EV_RPercentile', percentile_template],
    'T': ['Score', score_template]
}

for column in column_formats.keys():
    writer.sheets['Value Scoring'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Scoring'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

writer.save()

Saving in Excel...

